In [ ]:
#import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))



# notebooks/03_trading_simulation.ipynb

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from src.models.trading_engine import DynamicTradingEngine
from src.data_processor import DataProcessor

# Load data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')

# Initialize trading engine
engine = DynamicTradingEngine()

def get_season(date):
    month = pd.to_datetime(date).month
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

def simulate_weekly_trading(week_data, time_of_day=14):  # Default 2 PM
    # Calculate energy surplus/deficit for each household
    week_data['Delta'] = week_data['EnergyGeneratedFromRenewableSources'] - week_data['EnergyUsed']
    
    # Create sellers and buyers dataframes
    sellers = week_data[week_data['Delta'] > 0].copy()
    buyers = week_data[week_data['Delta'] < 0].copy()
    
    if len(sellers) == 0 or len(buyers) == 0:
        return pd.DataFrame()
    
    # Get season for the trading day
    season = get_season(week_data['Date'].iloc[0])
    
    # Prepare sellers dataframe with dynamic rates
    sellers_df = pd.DataFrame({
        'householdID': sellers['HouseholdID'],
        'unitsToBeSold': sellers['Delta'],
        'sellingRate': 0.14 * 0.8  # Initial rate, will be adjusted dynamically
    })
    
    # Prepare buyers dataframe with dynamic rates
    buyers_df = pd.DataFrame({
        'householdID': buyers['HouseholdID'],
        'demand': -buyers['Delta'],
        'buyingRate': 0.14  # Initial rate, will be adjusted dynamically
    })
    
    # Execute trades with dynamic pricing
    return engine.match_orders(sellers_df, buyers_df, time_of_day, season)

# Simulate trading for each week at different times
trading_times = [9, 14, 19]  # 9 AM, 2 PM, 7 PM
weekly_trades = []

for date in df['Date'].unique():
    week_data = df[df['Date'] == date]
    for time in trading_times:
        trades = simulate_weekly_trading(week_data, time)
        if not trades.empty:
            trades['Date'] = date
            trades['TimeOfDay'] = time
            weekly_trades.append(trades)

trading_history = pd.concat(weekly_trades, ignore_index=True)

# Visualizations
# 1. Trading Volume Over Time with Time-of-Day Breakdown
fig1 = go.Figure()
for time in trading_times:
    time_data = trading_history[trading_history['TimeOfDay'] == time]
    daily_volume = time_data.groupby('Date')['amount'].sum()
    fig1.add_trace(go.Scatter(
        x=daily_volume.index,
        y=daily_volume.values,
        name=f'{time}:00',
        mode='lines'
    ))
fig1.update_layout(
    title='Trading Volume by Time of Day',
    xaxis_title='Date',
    yaxis_title='Energy Traded (kWh)'
)
fig1.show()

# 2. Price Trends by Time of Day
fig2 = go.Figure()
for time in trading_times:
    time_data = trading_history[trading_history['TimeOfDay'] == time]
    daily_price = time_data.groupby('Date')['price_per_unit'].mean()
    fig2.add_trace(go.Scatter(
        x=daily_price.index,
        y=daily_price.values,
        name=f'{time}:00',
        mode='lines'
    ))
fig2.update_layout(
    title='Average Trading Price by Time of Day',
    xaxis_title='Date',
    yaxis_title='Price ($/kWh)'
)
fig2.show()

# 3. Seasonal Trading Analysis
trading_history['Season'] = trading_history['Date'].apply(get_season)
seasonal_stats = trading_history.groupby('Season').agg({
    'amount': 'sum',
    'price_per_unit': 'mean',
    'total_cost': 'sum'
}).round(2)

print("\nSeasonal Trading Statistics:")
print(seasonal_stats)

# 4. Household Trading Summary
household_summary = pd.DataFrame({
    'Total_Sales': trading_history.groupby('seller_id')['total_cost'].sum(),
    'Total_Purchases': trading_history.groupby('buyer_id')['total_cost'].sum(),
    'Volume_Sold': trading_history.groupby('seller_id')['amount'].sum(),
    'Volume_Bought': trading_history.groupby('buyer_id')['amount'].sum()
}).fillna(0).round(2)

print("\nHousehold Trading Summary:")
print(household_summary)

# 5. Trading Network Visualization (Last Day)
def plot_trading_network(trades_sample):
    network_fig = go.Figure()
    
    for _, trade in trades_sample.iterrows():
        network_fig.add_trace(go.Scatter(
            x=[trade['seller_id'], trade['buyer_id']],
            y=[trade['amount'], trade['amount']],
            mode='lines+markers',
            name=f"${trade['total_cost']:.2f}",
            text=[f"Seller: {trade['seller_id']}<br>Price: ${trade['price_per_unit']:.3f}/kWh", 
                  f"Buyer: {trade['buyer_id']}<br>Amount: {trade['amount']} kWh"],
            hoverinfo='text'
        ))
    
    network_fig.update_layout(
        title=f'Energy Trading Network (Time: {trades_sample["TimeOfDay"].iloc[0]}:00)',
        showlegend=False
    )
    return network_fig

last_date = trading_history['Date'].max()
for time in trading_times:
    last_trades = trading_history[
        (trading_history['Date'] == last_date) & 
        (trading_history['TimeOfDay'] == time)
    ]
    if not last_trades.empty:
        network_fig = plot_trading_network(last_trades)
        network_fig.show()


ImportError: cannot import name 'DynamicTradingEngine' from 'src.models.trading_engine' (c:\Spring 2025\hack-ncstate25\src\models\trading_engine.py)